Load data from results/ to compare Hill Climber and Evolutionary Strategy performance across all problem instances.

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from typing import Dict, Any, List, Union

In [9]:
# ====================== Setup and Helper Utilities ======================

# Assuming necessary utility and dataclasses are imported from src/
# from src.dataclasses.solution import SolutionResults

In [12]:
def decompress_rle(rle_history: Dict[str, List[Union[float, int]]]) -> np.ndarray:
    """
    Reconstructs the full fitness history array from Run-Length Encoding (RLE).
    RLE format: {'values': [v1, v2, ...], 'counts': [c1, c2, ...]}
    """
    values = rle_history.get('values', [])
    counts = rle_history.get('counts', [])
    if not values or not counts:
        return np.array([])
    
    # Use numpy.repeat to reconstruct the array
    return np.repeat(values, counts)

In [13]:
def load_all_results(dir_path: str = '../results/') -> Dict[str, Dict[str, List[Any]]]:
    """
    Loads all SolutionResults (.npy files) from the results directory
    and groups them by problem name and solver type.
    """
    all_results = {}
    
    for filename in os.listdir(dir_path):
        if filename.endswith('.npy'):
            parts = filename.split('_')
            problem_name = '_'.join(parts[1:-2]) # e.g., g_10
            solver_type = parts[-2]             # e.g., hc or es
            
            # Load the single SolutionResults object
            try:
                # np.load(..., allow_pickle=True).item() is often needed for single object
                result_obj = np.load(os.path.join(dir_path, filename), allow_pickle=True).item()
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue
            
            if problem_name not in all_results:
                all_results[problem_name] = {'hc': [], 'es': []}
                
            all_results[problem_name][solver_type].append(result_obj)
            
    return all_results

In [14]:
# ====================== Visualization Function ======================

In [15]:
def plot_comparison(problem_name: str, problem_data: Dict[str, List[Any]]):
    """
    Generates the 3-panel comparison plot for HC vs ES results.
    """
    hc_results = problem_data.get('hc', [])
    es_results = problem_data.get('es', [])
    
    if not hc_results and not es_results:
        print(f"No results available for {problem_name}")
        return

    # Basic Plot Setup
    fig, axes = plt.subplots(3, 1, figsize=(18, 15), sharex=False)
    fig.suptitle(f"Solver Performance Comparison: {problem_name}", fontsize=16)

    # --- Panel 1 (Upper): Hill Climber ---
    ax = axes[0]
    if hc_results:
        # Assuming only one HC run or taking the first one
        hc_history = decompress_rle(hc_results[0].history)
        ax.plot(hc_history, label='HC Fitness', color='black', alpha=0.7)
        ax.set_title("1. Hill Climber (Current vs Best Solution)")
        ax.set_ylabel("Fitness (Cost)")
    else:
        ax.set_title("1. Hill Climber (No Data)")
    ax.grid(True)

    # --- Panel 2 (Middle): ES (Greedy Init) ---
    ax = axes[1]
    greedy_es = [res for res in es_results if res.params.get('greedy_initial_solutions')]
    if greedy_es:
        for res in greedy_es:
            es_history = decompress_rle(res.history)
            label = (f"Mut={res.params.get('mutation_rate', '?')}, "
                     f"Pop={res.params.get('population_size', '?')}, "
                     f"Offs={res.params.get('offspring_size', '?')}")
            ax.plot(es_history, label=label, alpha=0.6)
        ax.set_title("2. Evolutionary Strategy (Greedy Initialization Overlay)")
        ax.set_ylabel("Fitness (Cost)")
        # ax.legend(loc='upper right', ncol=2, fontsize=8) # Omitted for simplicity
    else:
        ax.set_title("2. ES Greedy Init (No Data)")
    ax.grid(True)

    # --- Panel 3 (Lower): ES (Random Init) ---
    ax = axes[2]
    random_es = [res for res in es_results if not res.params.get('greedy_initial_solutions')]
    if random_es:
        for res in random_es:
            es_history = decompress_rle(res.history)
            label = (f"Mut={res.params.get('mutation_rate', '?')}, "
                     f"Pop={res.params.get('population_size', '?')}, "
                     f"Offs={res.params.get('offspring_size', '?')}")
            ax.plot(es_history, label=label, alpha=0.6)
        ax.set_title("3. Evolutionary Strategy (Random Initialization Overlay)")
        ax.set_xlabel("Iterations/Generations")
        ax.set_ylabel("Fitness (Cost)")
    else:
        ax.set_title("3. ES Random Init (No Data)")
    ax.grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [16]:
# ====================== Main Execution Block ======================

In [17]:
if __name__ == '__main__':
    # 1. Load Data
    print("Loading all benchmark results...")
    results_data = load_all_results()
    
    # 2. Generate Plots for each problem
    for problem_name, data in results_data.items():
        plot_comparison(problem_name, data)
        
    # 3. Final Summary (loading the final best fitness table)
    try:
        with open('../results/tsp_best_tuned.json', 'r') as f:
            final_best_solutions = json.load(f)
            
        print("\n================== Final Best Tuned Solutions ==================")
        # Note: In the actual notebook, this would be formatted as a nice table
        for name, data in final_best_solutions.items():
             print(f"{name}: Best Fitness = {data['best_fitness']:.4f}")
             
    except FileNotFoundError:
        print("\nWarning: 'tsp_best_tuned.json' not found. Run benchmarks first.")
        
    print("\nAnalysis notebook execution simulated.")

Loading all benchmark results...


Analysis notebook execution simulated.
